In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import yfinance as yf
from pandas_datareader import data as pdr
import warnings

yf.pdr_override() 
plt.style.use("fivethirtyeight")
warnings.filterwarnings("ignore")

yesterday = datetime.now() - timedelta(1)
yesterday = datetime.strftime(yesterday, '%Y-%m-%d')
today = datetime.strftime(datetime.now(), '%Y-%m-%d')

In [2]:
## FUNCTIONS
def getMonthlyData(ticker="SPY", start="1994-01-01", end="2022-10-01"):
    data = yf.download(ticker, start, end)
    return data.resample('M').ohlc()['Close'].reset_index()
    
def funcMomentum(data):
    data['shift1'] = data['close'].shift(1)
    data['shift3'] = data['close'].shift(3)
    data['shift6'] = data['close'].shift(6)
    data[['Date', 'close','shift1','shift3','shift6']]
    data = data.dropna(axis=0)
    data["momentum"] = (12*(data['close']/data['shift1']-1)) + (4*(data['close']/data['shift3']-1)) + (2*(data['close']/data['shift6']-1))
    return data


In [3]:
def acceleratingDualMomentum(stocks=["spy","scz","tip"]):
    ## Getting Data
    namespace = globals()
    for i in stocks:
        namespace['%s_data' % str(i)] = getMonthlyData(i, end=yesterday)

    ## Creating shift columns and calculating momentum
    for i in stocks:
        namespace['%s_data' % str(i)] = funcMomentum(namespace['%s_data' % str(i)])
        
   
    ## series lenght ## hardcoded because ETF release date won't change.
    date = scz_data['Date'] 
    spy_data2 = spy_data.copy()
    spy_data2 = spy_data2.tail(len(date))
    tip_data2 = tip_data.copy()
    tip_data2 = tip_data.tail(len(date))


    ## ETF Picking
    etf = []
    ret = []
    for i in range(0, len(date)):
        if ((spy_data2['momentum'].iloc[i] > scz_data['momentum'].iloc[i]) & (spy_data2['momentum'].iloc[i] > 0)):
            etf.append('SPY')
            ret.append(spy_data2['momentum'].iloc[i])
        elif ((scz_data['momentum'].iloc[i] > spy_data2['momentum'].iloc[i]) & (scz_data['momentum'].iloc[i] > 0)):
            etf.append('SCZ')
            ret.append(scz_data['momentum'].iloc[i])
        else:
            etf.append('TIP')
            ret.append(tip_data2['momentum'].iloc[i])
            
    ## Returns        
    ret_data = {'date':date, 'etf':etf, 'ret':ret, 'Cumulative Returns':np.cumsum(ret)}
    returns = pd.DataFrame(data=ret_data)
    return returns

In [5]:
adm = acceleratingDualMomentum()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
stocks=["spy","scz","tip"]
## Getting Data
namespace = globals()
for i in stocks:
    namespace['%s_data' % str(i)] = getMonthlyData(i, end=yesterday)

## Creating shift columns and calculating momentum
for i in stocks:
    namespace['%s_data' % str(i)] = funcMomentum(namespace['%s_data' % str(i)])

In [ ]:
## series lenght ## hardcoded because ETF release date won't change.
date = scz_data['Date'] 
spy_data2 = spy_data.copy()
spy_data2 = spy_data2.tail(len(date))
tip_data2 = tip_data.copy()
tip_data2 = tip_data.tail(len(date))


## ETF Picking
etf = []
ret = []
for i in range(0, len(date)):
    if ((spy_data2['momentum'].iloc[i] > scz_data['momentum'].iloc[i]) & (spy_data2['momentum'].iloc[i] > 0)):
        etf.append('SPY')
        ret.append(spy_data2['momentum'].iloc[i])
    elif ((scz_data['momentum'].iloc[i] > spy_data2['momentum'].iloc[i]) & (scz_data['momentum'].iloc[i] > 0)):
        etf.append('SCZ')
        ret.append(scz_data['momentum'].iloc[i])
    else:
        etf.append('TIP')
        ret.append(tip_data2['momentum'].iloc[i])

In [ ]:
## Returns        
ret_data = {'date':date, 'etf':etf, 'ret':ret, 'Cumulative Returns':np.cumsum(ret)}
returns = pd.DataFrame(data=ret_data)
returns